# TO RUN
1. Make sure you installed all the dependencies
2. open **'init_notebook.py'** and change the **projectFolder** to the src folder of the repository.
3. In the following cell change the dataDir to the data directory of the InD dataset

In [1]:
exec(open('init_notebook.py').read())

current working dirE:\AV\DataSetExploration\drone-dataset-tools\src


In [2]:
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig(level=logging.INFO)
from loguru import logger
logger.remove()
logger.add(sys.stdout, format="{level} {message}", filter="my_module", level="INFO")

In [4]:
# Unit tests on preprocessed scene data of location 2

In [5]:
from extractors.LocationData import LocationData
scene10 = LocationData.load("../data/location-2", "2022-11-13-fps-2.5-scene-10.dll")

In [15]:
allPedDfs = scene10.getClippedPedDfs()
otherDfs = scene10.getClippedOtherDfs()
allPedIds = scene10.uniqueClippedPedIds()
allOtherIds = scene10.uniqueClippedOtherIds()

# pedClasses = allPedDfs['class'].unique()
otherClasses = otherDfs['class'].unique()

In [13]:
assert len(otherClasses) > 1
assert 'pedestrian' not in otherClasses

In [16]:
assert allPedIds == allPedDfs["uniqueTrackId"].unique()
assert allOtherIds == otherDfs["uniqueTrackId"].unique()

In [91]:
primaryPedId = allPedIds[0]
primaryDf = scene10.getClippedPedDfByUniqueTrackId(primaryPedId).copy()
primaryPedId

18023

In [92]:
from extractors.PedScenarioBuilder import PedScenarioBuilder
scenarioBuilder = PedScenarioBuilder()

In [93]:
recordingId, start, end, roadWidth = scenarioBuilder.getRecordStartEndWidth(primaryDf)

In [94]:
assert primaryDf["frame"].min() == start
assert primaryDf["frame"].max() == end
assert len(primaryDf["recordingId"].unique()) == 1
assert primaryDf["recordingId"].unique()[0] == recordingId
assert len(primaryDf["roadWidth"].unique()) == 1
assert primaryDf["roadWidth"].unique()[-1] == roadWidth

In [96]:
anotherDf = otherDfs[otherDfs['uniqueTrackId'] == allOtherIds[0]]

In [104]:
scenePeds = {}

scenePeds[18107] = [18112] #6593 6723
scenePeds[18112] = [18107]
scenePeds[18127] = [18134]
scenePeds[18134] = [18127]


In [105]:
# scenarioBuilder.printSceneStats(scene10)

In [109]:

for primaryPedId in scenePeds:
    primaryDf = scene10.getClippedPedDfByUniqueTrackId(primaryPedId).copy()
    recordingId, start, end, roadWidth = scenarioBuilder.getRecordStartEndWidth(primaryDf)
    scenarioPedTracks = scenarioBuilder.getOtherPedScenarioTracksFromScene(
        primaryDf = primaryDf,
        allPedDfs=allPedDfs,
        recordingId = recordingId,
        start=start,
        end=end
    )
    # print("primaryPedId", primaryPedId)
    # print(scenePeds[primaryPedId])
    # print(scenarioPedTracks["uniqueTrackId"].unique())
    assert scenarioPedTracks["uniqueTrackId"].unique() == scenePeds[primaryPedId];
    assert len(scenarioPedTracks["uniqueTrackId"].unique()) == len(scenePeds[primaryPedId]);

In [112]:

for primaryPedId in scenePeds:
    primaryDf = scene10.getClippedPedDfByUniqueTrackId(primaryPedId).copy()
    recordingId, start, end, roadWidth = scenarioBuilder.getRecordStartEndWidth(primaryDf)
    scenarioOtherTracks = scenarioBuilder.getOtherScenarioTracksFromScene(
        otherSceneDf=otherDfs,
        recordingId = recordingId,
        start=start,
        end=end
    )
    print("primaryPedId", primaryPedId)
    print(scenarioOtherTracks["uniqueTrackId"].unique())

primaryPedId 18107
<IntegerArray>
[18111, 18113, 18105]
Length: 3, dtype: Int64
primaryPedId 18112
<IntegerArray>
[18111, 18113, 18105]
Length: 3, dtype: Int64
primaryPedId 18127
<IntegerArray>
[]
Length: 0, dtype: Int64
primaryPedId 18134
<IntegerArray>
[]
Length: 0, dtype: Int64
